In [1]:
!pip install pandas

In [2]:
import pandas as pd

df = pd.read_csv("../data/bacteria/IDB_interesting_bacteria.csv")
df.head()

,ID,bacteria,isolate,link,Tax ID
0,90,Bacteria;Firmicutes;Clostridia;Clostridiales;L...,NaN,https://www.genome.jp/dbget-bin/www_bfind_sub?...,NaN
1,90,Bacteria;Firmicutes;Clostridia;Clostridiales;L...,T02583,https://www.genome.jp/entry/gn:T02583,751585.0
2,90,Bacteria;Firmicutes;Clostridia;Clostridiales;L...,T02584,https://www.genome.jp/entry/gn:T02584,717962.0
3,90,Bacteria;Firmicutes;Clostridia;Clostridiales;L...,T07118,https://www.genome.jp/entry/gn:T07118,410072.0
4,90,Bacteria;Firmicutes;Clostridia;Clostridiales;L...,T07943,https://www.genome.jp/entry/gn:T07943,411474.0


In [3]:
isolate_list = df.isolate.dropna().tolist()
print(len(isolate_list))
print(isolate_list)

23
['T02583', 'T02584', 'T07118', 'T07943', 'T01387', 'T02600', 'T02601', 'T02606', 'T03158', 'T05998', 'T08490', 'T08757', 'T00835', 'T07161', 'T08965', 'T03807', 'T05838', 'T06636', 'T09392', 'T07161', 'T08965', 'T04880', 'T08068']


In [4]:
def split_to_pd(s):
    lines = s.split("\n")
    all_lines = [aline.split("\t") for aline in lines]
    df_gene_list = pd.DataFrame(all_lines, columns=['A', 'B', 'C', 'D'])
    return df_gene_list

In [5]:
import requests

all_genes_list = []

# for i in isolate_list[0:2]:
for i in isolate_list:
    url = 'https://rest.kegg.jp/list/' + i
    gene_list = requests.get(url)
    print(url, gene_list)
    all_genes_list.append(split_to_pd(gene_list.text))
#     print(gene_list.text)
#print(response_API.status_code)

https://rest.kegg.jp/list/T02583 <Response [200]>
https://rest.kegg.jp/list/T02584 <Response [200]>
https://rest.kegg.jp/list/T07118 <Response [200]>
https://rest.kegg.jp/list/T07943 <Response [200]>
https://rest.kegg.jp/list/T01387 <Response [200]>
https://rest.kegg.jp/list/T02600 <Response [200]>
https://rest.kegg.jp/list/T02601 <Response [200]>
https://rest.kegg.jp/list/T02606 <Response [200]>
https://rest.kegg.jp/list/T03158 <Response [200]>
https://rest.kegg.jp/list/T05998 <Response [200]>
https://rest.kegg.jp/list/T08490 <Response [200]>
https://rest.kegg.jp/list/T08757 <Response [200]>
https://rest.kegg.jp/list/T00835 <Response [200]>
https://rest.kegg.jp/list/T07161 <Response [200]>
https://rest.kegg.jp/list/T08965 <Response [200]>
https://rest.kegg.jp/list/T03807 <Response [200]>
https://rest.kegg.jp/list/T05838 <Response [200]>
https://rest.kegg.jp/list/T06636 <Response [200]>
https://rest.kegg.jp/list/T09392 <Response [200]>
https://rest.kegg.jp/list/T07161 <Response [200]>


In [6]:
len(all_genes_list)

23

In [7]:
all_genes_list

[                    A     B                             C  \
 0     coo:CCU_T_29320  tRNA                       60..130   
 1       coo:CCU_00100   CDS                      241..318   
 2       coo:CCU_00110   CDS                     432..1544   
 3       coo:CCU_00120   CDS                    1547..3220   
 4       coo:CCU_00140   CDS                    5775..5873   
 ...               ...   ...                           ...   
 2158    coo:CCU_29220   CDS  complement(3113670..3114398)   
 2159    coo:CCU_29230   CDS  complement(3114417..3114926)   
 2160    coo:CCU_29240   CDS  complement(3115005..3115157)   
 2161    coo:CCU_29270   CDS  complement(3117965..3119641)   
 2162                   None                          None   
 
                                                       D  
 0                                           CCU_T_29320  
 1                                  hypothetical protein  
 2                                DNA repair exonuclease  
 3                

In [8]:
def find_intersect(data_1, data_2):
    intersect = list(set(data_1) & set(data_2))
    return intersect

def find_union(data_1, data_2):
    union = list(set(data_1) | set(data_2))
    return union

In [9]:
base = list()
for i in range(len(all_genes_list)):
    base = find_union(base, all_genes_list[i].D.tolist())
len(base)

11690

In [11]:
base = all_genes_list[0].D.tolist()

for i in range(1, len(all_genes_list) -1):
    temp_intersection = find_intersect(all_genes_list[i].D.tolist(), all_genes_list[i+1].D.tolist())

    #     print(temp_intersection)
    base = find_intersect(base, temp_intersection)
    print(base)

    print(i, i+1, len(base))

['dephospho-CoA kinase', 'UDP-N-acetylmuramoyl-tripeptide--D-alanyl-D-alanine ligase', 'methylglyoxal synthase', 'shikimate dehydrogenase', 'argininosuccinate synthase', 'peptide deformylase', 'transcriptional regulator', 'adenine phosphoribosyltransferase', 'acetylglutamate kinase', 'anti-sigma F factor', 'trigger factor', 'amidophosphoribosyltransferase', 'glutamate-5-semialdehyde dehydrogenase', "deoxyuridine 5'-triphosphate nucleotidohydrolase", 'DNA gyrase subunit B', 'ACT domain-containing protein', 'methionine adenosyltransferase', 'guanylate kinase', 'hypothetical protein', None, 'thioredoxin', 'D-tyrosyl-tRNA(Tyr) deacylase', 'CTP synthase', 'N-acetylmuramoyl-L-alanine amidase', 'SAM-dependent methyltransferase', 'dihydroxy-acid dehydratase', '2,3-bisphosphoglycerate-independent phosphoglycerate mutase', 'UDP-N-acetylmuramate--L-alanine ligase', 'adenylosuccinate lyase', 'cysteine desulfurase', 'threonine synthase', 'glucose-6-phosphate isomerase', '6,7-dimethyl-8-ribityllumaz

In [18]:
import pandas

df = pd.read_csv('../data/bacteria/13059_2022_2643_MOESM7_ESM.csv')
bac_genes = df.bacterial_gene_names
bac_genes = list(set(bac_genes))
bac_genes

["['uxuA YPK_2778' 'uxuA Tola_2884' nan]",
 "['gap gapC CA_C0709' 'gapA gap MG301' nan]",
 "['aroE YE3884' 'aroE EcSMS35_3577' nan]",
 "['glpK LCA_0649' 'glpK BcerKBAB4_0948' nan]",
 "['FAEPRAA2165_01342' 'CRH10_03100' nan]",
 "['ppaC BAA_2888' 'ppaC BPUM_3708' nan]",
 "['nusA c3926' 'nusA Z4530 ECs4050' nan]",
 "['astD EcE24377A_1968' 'astD CPS_0634' nan]",
 "['trhP1 yrrO BSU27340' 'SAMN05878482_101721' nan]",
 "['pgcA gtaC gtaE yhxB BSU09310' 'SC09_Contig19orf00130' nan]",
 "['SAMN02799616_04514' 'DN390_09215' nan]",
 "['pntA b1603 JW1595' 'A1U5_01956' nan]",
 "['fklB VME_16270' 'fklB XSR1_90091' nan]",
 "['atpD BMA10229_A1587' 'atpD Bcep1808_0115' nan]",
 "['metE BCB4264_A4108' 'metE BCAH820_4021' nan]",
 "['gltX Xfasm12_2028' 'gltX YPDSF_2092' nan]",
 "['KS08_01620' 'D9V42_07860' nan]",
 "['eno SUB0655' 'eno EAT1b_0860' nan]",
 "['fib efb MW1040' 'fib efb SAS1091' nan]",
 "['AT03_04030' 'alcB YPH_3898' nan]",
 "['clpB BC_1168' 'clpB CPE1428' nan]",
 "['HV084_25475' 'HV269_27150' na

In [24]:
for bac in bac_genes:
    print(bac, bac.strip('[]'))

['uxuA YPK_2778' 'uxuA Tola_2884' nan] 'uxuA YPK_2778' 'uxuA Tola_2884' nan
['gap gapC CA_C0709' 'gapA gap MG301' nan] 'gap gapC CA_C0709' 'gapA gap MG301' nan
['aroE YE3884' 'aroE EcSMS35_3577' nan] 'aroE YE3884' 'aroE EcSMS35_3577' nan
['glpK LCA_0649' 'glpK BcerKBAB4_0948' nan] 'glpK LCA_0649' 'glpK BcerKBAB4_0948' nan
['FAEPRAA2165_01342' 'CRH10_03100' nan] 'FAEPRAA2165_01342' 'CRH10_03100' nan
['ppaC BAA_2888' 'ppaC BPUM_3708' nan] 'ppaC BAA_2888' 'ppaC BPUM_3708' nan
['nusA c3926' 'nusA Z4530 ECs4050' nan] 'nusA c3926' 'nusA Z4530 ECs4050' nan
['astD EcE24377A_1968' 'astD CPS_0634' nan] 'astD EcE24377A_1968' 'astD CPS_0634' nan
['trhP1 yrrO BSU27340' 'SAMN05878482_101721' nan] 'trhP1 yrrO BSU27340' 'SAMN05878482_101721' nan
['pgcA gtaC gtaE yhxB BSU09310' 'SC09_Contig19orf00130' nan] 'pgcA gtaC gtaE yhxB BSU09310' 'SC09_Contig19orf00130' nan
['SAMN02799616_04514' 'DN390_09215' nan] 'SAMN02799616_04514' 'DN390_09215' nan
['pntA b1603 JW1595' 'A1U5_01956' nan] 'pntA b1603 JW1595' '